In [5]:
import numpy as np
import matplotlib.pyplot as plt

In [73]:
def choose_initial_cluster_centers(points, clusters):
    us = np.zeros((clusters,784))         # centers of each cluster
    #print(us.shape)
    indicies = np.array(clusters)

    index = np.random.choice(points.shape[0], 1)[0]         # choose first point randomly
    indicies = np.append(indicies, index)
    us[0] =  points[index]

    for i in range(1,clusters):
        max_distance = 0
        for j in range(points.shape[0]):    # choose the center with the biggest distancce form last center
            if j not in indicies:           # check if the the point is already a center to cluster
                point = np.array(points[j])
                last_point = us[i-1]
                distance = np.linalg.norm(point-last_point)     # Eucledian distance

                if distance > max_distance:
                    max_distance = distance
                    index = j
                    u = point
        
        
        indicies = np.append(indicies, index)
        us[i] = u
    #print(us.shape)
    return us

In [85]:
def compute_membership(centers, point, rnks):
    #print(centers.shape[0])
    min_distance = np.linalg.norm(point-centers[0])
    index = 0
    for i in range(1,centers.shape[0]):
        
        center = centers[i]
        if(point.all() != center.all()):
            distance = np.linalg.norm(point-center)

        if distance < min_distance and distance > 0:
            min_distance = distance
            index = i
        elif distance == min_distance and distance > 0:      # if the distances are the same between two centers, choose the one with more members
            if rnks[i] > rnks[index]:
                index = i
    print(distance)
    return index, min_distance

In [83]:
def kmeans(points, clusters):

    old_centers = np.zeros(clusters)
    centers = choose_initial_cluster_centers(points, clusters)
    points_sum = np.zeros((10,784))
    distances = np.zeros(clusters)  # sum of distances in each cluster from the points to the cluster centers

    
    for limit in range(1000):
        if centers.all() == old_centers.all():      # stop if the algorithm converges
            break
        print("Iteration Number : ")
        print(limit)
        points_sum = np.zeros((10,784))
        rnks = np.zeros(clusters)       # used for counting number of members in each cluster
        distances = np.zeros(clusters)  # sum of distances in each cluster from the points to the cluster centers
        old_centers = centers

        for i in range(points.shape[0]):
            point = points[i]
            rnk, distance = compute_membership(centers, point, rnks)    # returns the cluster number of the point and its distance from the mean of the cluster
            rnks[rnk] += 1
            distances[rnk] += distance
            points_sum[rnk] += point
        
        for i in range(clusters): 
            centers[i] = points_sum[i] / rnks[i]         # Adjust the cluster centers
    #print(distances)
    sum_distances = np.sum(distances)
    
    return centers, sum_distances

In [10]:
images = []

for i in range(2400):
    img = plt.imread("Images/" + str((i+1)) + ".jpg")
    pixels = []
    for j in range(28):
        for k in range(28):
            pixels.append(img[j,k])
    
    images.append(pixels)
    
images = np.asarray(images)
print(images.shape)

(2400, 784)


In [11]:
imagesT = np.transpose(images)
print(imagesT.shape)

(784, 2400)


In [12]:
labels = np.genfromtxt('Images/Training Labels.txt')
print(labels)

[0. 0. 0. ... 9. 9. 9.]


In [86]:
centers_arr = np.zeros((30,10,784))
distances_arr = np.zeros(30)
for i in range(30):
    centers_arr[i], distances_arr[i] = kmeans(images,10)
print(distances_arr)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
